타이타닉

타이타닉 데이터로 생존율 측정 \
살았는지 죽었는지를 label로 잡고, \
성별, 나이 등 feature로 선택, \
데이터 Null 값 확인, 문자열이 있으면 숫자로 바꾸고 (label, OneHot 인코딩), 이 데이터 정규화 되어 있는지 확인 해보고 Kfold 등으로 검증셋 만들고 알고리즘 돌리기. \
GridSearchCV는 안함 (max.. 가 뭔지 모르기 때문에~) \
Cross_Val_score() 넣어서 교차검증까지 하고 학습시키기 \
평가는 accuracy, accuracy_score

1. pandas이용해서 csv파일 읽어오기

In [2]:
# 파이썬 타이타닉 csv 파일로 데이터 가져오기
# 구글 검색
# https://www.kaggle.com/competitions/titanic/data
# 캐글에서 csv 파일 다운 받고 

# pandas를 이용해서 csv파일 읽어오기
import pandas as pd

df_titanic = pd.read_csv('../ML/datasets/titanic_train.csv')
df_titanic.head()

FileNotFoundError: [Errno 2] No such file or directory: '../ML/datasets/titanic_train.csv'

In [ ]:
# 데이터 째려보기1
df_titanic.info()

# age에 null 값이 있다면, ffill이나 bfill이나 평균값을 넣어줄 수 있을 것
# Dtype 데이터 타입을 봐야함, Sex가 object로 되어 있는건 글자로 되어 있는 것

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# 데이터 째려보기2
df_titanic.describe()

# Survived를 label로 쓸 것
# Age가 표준편차 std가 크니, 스케일링을 해줄 수도 있을 것

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# feature로 ['Pclass', 'Sex', 'Age'] 사용
# 이외의 행 드랍시키고 df로 만들어줌

df_titanic = df_titanic.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Fare', 'Ticket', 'Cabin', 'Embarked'], axis=1)
df_titanic.head()

,Survived,Pclass,Sex,Age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0


In [ ]:
# loc를 이용해 행 드랍 (iloc는 인덱스로 불러오기 때문에 컬럼명으로 지정은 loc 써야함)
df_titanic = df_titanic.loc[:, ['Survived', 'Pclass', 'Sex', 'Age']]
df_titanic.head()

,Survived,Pclass,Sex,Age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0


In [ ]:
# feature 값 X로 지정
X = df_titanic.drop('Survived', axis=1)
# X = df_titanic.loc[:, ['Pclass', 'Sex', 'Age']] 이렇게 해도 됨
X

,Pclass,Sex,Age
0,3,male,22.0
1,1,female,38.0
2,3,female,26.0
3,1,female,35.0
4,3,male,35.0
...,...,...,...
886,2,male,27.0
887,1,female,19.0
888,3,female,NaN
889,1,male,26.0


In [ ]:
# label 값 지정
y = df_titanic['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

3. 결측치 처리

In [ ]:
# feature 값의 결측치 처리
# ffill을 이용해서 처리
X.isna().sum()

Pclass      0
Sex         0
Age       177
dtype: int64

In [ ]:
X['Age'] = X['Age'].fillna(method = 'ffill')
# X['Age'].fillna(method = 'ffill', inplace=True)
X.isna().sum()

Pclass    0
Sex       0
Age       0
dtype: int64

4. Sex를 label encoding 해줘야 함 \
male, female로 되어 있었음

In [ ]:
X.info()
# Sex가 object라 바꿔줘야함 (문자열)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    object 
 2   Age     891 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB


In [ ]:
# 이렇게 해도 되긴 함함
pd.get_dummies(X)

,Pclass,Age,Sex_female,Sex_male
0,3,22.0,0,1
1,1,38.0,1,0
2,3,26.0,1,0
3,1,35.0,1,0
4,3,35.0,0,1
...,...,...,...,...
886,2,27.0,0,1
887,1,19.0,1,0
888,3,19.0,1,0
889,1,26.0,0,1


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
labels = encoder.fit_transform(X['Sex'])
X['Sex'] = labels
X.head()

,Pclass,Sex,Age
0,3,1,22.0
1,1,0,38.0
2,3,0,26.0
3,1,0,35.0
4,3,1,35.0


5. feature Scailing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
age = np.array(X['Age']).reshape(-1, 1)
scaler.fit(age)
age = scaler.transform(age)
X['Age'] = age

X.describe()

,Pclass,Sex,Age
count,891.000000,891.000000,891.000000
mean,2.308642,0.647587,0.366443
std,0.836071,0.477990,0.182893
min,1.000000,0.000000,0.000000
25%,2.000000,0.000000,0.246042
50%,3.000000,1.000000,0.346569
75%,3.000000,1.000000,0.472229
max,3.000000,1.000000,1.000000


6. DecisionTreeClassifier와 cross_val_score 활용해 학습

In [ ]:
# cv = 5
# X : features
# y : label

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

dt_clf = DecisionTreeClassifier(random_state=121)
scores = cross_val_score(dt_clf, X, y, scoring='accuracy', cv=5)

print('교차 검증별 정확도 : ', np.round(scores, 4)) 
print('평균 검증 정확도 : ', np.round(np.mean(scores), 4))

교차 검증별 정확도 :  [0.7933 0.7865 0.7865 0.7697 0.809 ]
평균 검증 정확도 :  0.789


In [ ]:
scores

array([0.79329609, 0.78651685, 0.78651685, 0.76966292, 0.80898876])

In [ ]:
scores.mean() 
# 정확도가 0.78이면 좋은거라 할 수 있지만, 이 데이터에서 이정도면 안좋은 것
# X = X.drop('Age', axis=1) 를 해도 정확도값이 0.77 나옴

0.7889962965287804

## 여기 이후 내용 너무 연연해하지 말기

7. 예측 및 평가

In [ ]:
# predict을 위한 titanic_test 데이터 전처리 및 scaling

# 데이터 불러오기
df_titanic_test = pd.read_csv('./datasets/titanic/test.csv')

# feature 선택
X_test = df_titanic_test.loc[:, ['Pclass', 'Sex', 'Age']]

# age 결측치 처리
X_test['Age'] = X_test['Age'].fillna(method = 'ffill')

# labelencoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels = encoder.fit_transform(X_test['Sex'])
X_test['Sex'] = labels

# MinMaxScaler 활용해 age 정규화
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler = MinMaxScaler()
age = np.array(X_test['Age']).reshape(-1, 1)
scaler.fit(age)
age = scaler.transform(age)
X_test['Age'] = age

X_test.describe()

,Pclass,Sex,Age
count,418.000000,418.000000,418.000000
mean,2.265550,0.636364,0.395028
std,0.841838,0.481622,0.183379
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.287881
50%,3.000000,1.000000,0.353818
75%,3.000000,1.000000,0.512066
max,3.000000,1.000000,1.000000


In [ ]:
# predict 학습.predict(테스트데이터)
pred = dt_clf.predict(X_test)

# 이렇게 하니까 오류가 나서 cross 하면서 predict도 같이 보여주는 라이브러리를 사용하기로 

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

### sklearn.model_selection.cross_val_predict
* sklearn.model_selection.cross_val_predict(estimator, X, y=None, *, groups=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', method='predict')
> https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html

In [ ]:
from sklearn.model_selection import cross_val_predict

pred = cross_val_predict(dt_clf, X, y, cv=5)
# 음... 모르겠다

In [ ]:
# 이 위에 위에 오류가 난건 fit을 안했기 때문,
# 이 오류를 고쳐서 예측 값을 보기 위해 fit 해주기로
dt_clf.fit(X, y)
pred = dt_clf.predict(X_test)

pred

# X_test 자체에는 label 값이 없어서 테스트가 안됨
# 예측만 하고 평가는 안한거로~
# 평가는 0.78에서 다 한거로 ㅇㅇ
# grid_search_cv하면 편한데.. 최적의 알고리즘을 전달해주니까

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,

8. GridSearchCV 활용

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=121)
parameters = {'max_depth' : [1, 2, 3], 'min_samples_split' : [2, 3]}

grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, cv=5, refit=True) 
# 여기서 fit할 때 학습하면서 cross validation 해주면서 최적의 값을 찾아서 그 값을 grid_dtree에 저장해둠
# best_df_clf = grid_dtree.best_estimator_ 이렇게 저장해주면서...
# cross_val_score로 다시 한 번 훈련시킨 다음에 해도 상관 없지만..
# 평가는 지금 할 수 없음
grid_dtree.fit(X, y) # train이 X

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=121),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

In [ ]:
grid_dtree.best_estimator_
# 최적의 파라미터 값을 찾아줌

DecisionTreeClassifier(max_depth=1, random_state=121)

In [ ]:
best_df_clf = grid_dtree.best_estimator_
# best 값을, 최적의 하이퍼 파라미터값 찾은 걸 저장해줘

In [ ]:
best_df_clf.predict(X_test)
# 그리고 그걸 predict 해줘

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

# 데이터 다시 받아서 다시~ 예측 및 평가~

In [ ]:
pred = best_df_clf.predict(X_test)
len(pred)

418

In [ ]:
from sklearn.metrics import accuracy_score
import pandas as pd

y_df = pd.read_csv('./datasets/titanic/gender_submission.csv')
y_df.head()
#accuracy_score()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [ ]:
y_test = y_df['Survived']

In [ ]:
accuracy_score(y_test, pred)
# 100%가 나와????? 이건 말이 안됨

1.0

In [ ]:
dt_clf.fit(X, y)
pred_1 = dt_clf.predict(X_test)
len(pred_1)

418

In [ ]:
accuracy_score(y_test, pred_1)

0.8301435406698564